In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import time
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
# 加载ad_static_feature和ad_operation数据
ad_static_feature_df = pd.read_csv('ad_static_feature.csv')
ad_operation_df = pd.read_csv('ad_operation.csv')
#ad_static_feature_df.info()

C:\Users\USER\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
f = open(r'testA/totalExposureLog.out')
feature_names = ['Ad_Request_id', 'Ad_Request_Time', 'Ad_pos_id', 'user_id', 'ad_id', 
                 'Ad_material_size','Ad_bid', 'Ad_pctr', 'Ad_quality_ecpm', 'Ad_total_Ecpm']

totalExpLog = pd.read_csv(f, sep='\t',names=feature_names,engine = 'python',iterator=True)
loop = True
chunkSize = 1000000
chunks = []
index=0
while loop:
    try:
        # 分批处理
        chunk = totalExpLog.get_chunk(chunkSize)
        # 添加年月日特征
        chunk['year'] =chunk['Ad_Request_Time'].apply(lambda x: time.localtime(x).tm_year)
        chunk['month'] = chunk['Ad_Request_Time'].apply(lambda x: time.localtime(x).tm_mon)
        chunk['day'] =chunk['Ad_Request_Time'].apply(lambda x: time.localtime(x).tm_mday)
        # 时间戳转化为时间
        chunk['Ad_Request_Time'] = chunk['Ad_Request_Time'].apply(lambda x:time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(x))) 
        
        # 清洗exposure_log数据
        # 删除不合理时间
        chunk = chunk[~chunk['Ad_Request_Time'].isin(['2019-02-30 00:00:00'])]
        chunk = chunk[~chunk['Ad_Request_Time'].isin(['2019-02-29 00:00:00'])]
        # 删除空数据
        chunk = chunk.dropna()
        # 删除广告位id出现的多值数据
        chunk = chunk[~chunk['Ad_pos_id'].isin([','])]
        # 清洗掉在ad_static_feature中不存在的广告id
        chunk = pd.merge(chunk,ad_static_feature_df.loc[:,['ad_id']],how='inner',on='ad_id')
        # 清洗掉在ad_operation中不存在的广告id
        chunk = pd.merge(chunk,ad_operation_df.loc[:,['ad_id']],how='inner',on='ad_id')
        # 去重
        chunk = chunk.drop_duplicates(keep='first', inplace=False)
        # 重置索引
        chunk = chunk.reset_index(drop=True)
        chunks.append(chunk)
        index+=1

    except StopIteration:
        loop = False
        print("Iteration is stopped.")
print('开始合并')
totalExpLog = pd.concat(chunks, ignore_index= True)
print('合并结束')
# 去重
totalExpLog = totalExpLog.drop_duplicates(["Ad_Request_id","Ad_Request_Time","Ad_pos_id",
                                          "user_id","ad_id","Ad_material_size"],
                                          keep='first', inplace=False).reset_index(drop=True)
#totalExpLog.info()
totalExpLog.to_csv('totalExpLog.csv', index=False)